# Поиск выбросов (outliers)

## Поиск по порогу
При погружении в предметную область вы начинаете понимать, какие значения данные признаки могут принимать, а какие - не могут. Основываясь на этом выделяем выбросы.

## Поиск по отклонению (z-score)

Есть еще один способ поиска выбросов, не требующий настолько же пристального взгляда на данные, что и предыдущий метод.  
Он основан на поиске значения, на которое отклоняется данная точка от среднего. Делается это с помощью z-score, то есть, расстояния (измеряемого в количестве стандартных отклонений) от среднего значения точек.  

In [9]:
def outliers_z_score(x, threshold=3):
    mean = np.mean(x)
    std = np.std(x)
    z_scores = [(element - mean_y) / std for element in x]
    return np.where(np.abs(z_scores) > threshold)[0]

Значение по умолчанию для функции поиска выбросов равно `3`. Это означает, что мы считаем выбросами все, что находится на расстоянии больше 3 стандартных отклонений от среднего. Если мы подвигаем этот параметр в большую или меньшую сторону, мы найдем меньше или больше выбросов.

## IQR (interquartile range, межквартильное расстояние)

Обнаружение и обработка выбросов, используя метод **IQR** (interquartile range, межквартильное расстояние)

Строим `boxplot` и все значения вне "усов" считаем выбросами. Левая линия ящика обозначает первый квартиль (25-й перцентиль q25), а правая линия ящика обозначает третий квартиль (75-й перцентиль q75). Границы "усов" определяются так: 

нижняя граница - это минимальное значение между q25 - (1.5*iqr) и q25,

верхняя граница  - это максимальное значение между q75 и q75 + (1.5*iqr), 

где  iqr = q75-q25

In [10]:
import seaborn as sns

In [11]:
def outliers_iqr(x):
    quartile_1, quartile_3 = np.percentile(x, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((x > upper_bound) | (x < lower_bound))[0]

## Поиск по распределению

Еще один способ найти выбросы - построить нормальное распределение поверх данных, посчитать вероятность находиться в данном распределении для каждого значения. Самые невероятные значения будем считать выбросами.

In [7]:
from scipy.stats import norm

def estimate_gaussian(dataset):
    mu = np.mean(dataset, axis=0)
    sigma = np.cov(dataset.T)
    return mu, sigma
    
def get_gaussian(mu, sigma):
    distribution = norm(mu, sigma)
    return distribution

def get_probs(distribution, dataset):
    return distribution.pdf(dataset)

In [12]:
mu, sigma = estimate_gaussian(df['...'])
distribution = get_gaussian(mu, sigma)
probabilities = get_probs(distribution, df['...'])

NameError: name 'df' is not defined

Мы построили нормальное распределение на предоставленных данных и посчитали вероятность появления данной точки при данном распределении для каждого значения. Теперь выберем разумное значение вероятности, ниже которой будем считать точку выбросом. Например, `0.014`:

In [22]:
o_1 = np.where(probabilities < 0.014)[0]